<a href="https://colab.research.google.com/github/cxbxmxcx/Practical_AI_on_GCP/blob/master/Chapter_6_Face_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
!pip install face_recognition
!pip install --upgrade youtube_dl

In [ ]:
from __future__ import unicode_literals
import youtube_dl

video_url = 'https://www.youtube.com/watch?v=DHdkpSAteG0'

download_options = {}
download = youtube_dl.YoutubeDL(download_options)
info_dict = download.extract_info(video_url, download=False)
formats = info_dict.get('formats',None)

for f in formats:    
  if f.get('format_note',None) == '144p':        
    url = f.get('url',None)  

print(url)

In [ ]:
import face_recognition
import cv2

input_movie = cv2.VideoCapture(url)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

In [ ]:
%%bash
git clone https://github.com/gurugaurav/bing_image_downloader
cd bing_image_downloader
pip install .

In [ ]:
from bing_image_downloader import downloader
keywords = "will ferrel"
downloader.download("will ferrel", limit=100, adult_filter_off=False, force_replace=True)

In [ ]:
path = '/content/dataset/bing/will ferrel'
import os
from os import listdir
from os.path import isfile, join
files = [path + '/' + f for f in listdir(path) if isfile(join(path, f))]
print(files)

encoded_faces = []
for file in files:
  if os.path.isfile(file):
    print('Encoding image - ' + file)
    img = face_recognition.load_image_file(file)
    img_enc = face_recognition.face_encodings(img)[0]
    encoded_faces.append(img_enc)

print(encoded_faces[0])

In [ ]:
from google.colab.patches import cv2_imshow

face_locations = []
face_encodings = []
face_names = []
frame_number = 0

while True:
    ret, frame = input_movie.read()
    frame_number += 1

    if not ret:
        break
    
    rgb_frame = frame[:, :, ::-1]

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:        
        match = face_recognition.compare_faces(encoded_faces, face_encoding, tolerance=0.50)
        face_names.append(keywords)

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
    
    cv2_imshow(frame)
input_movie.release()
cv2.destroyAllWindows()